<a href="https://colab.research.google.com/github/parisa-kavian/AI-Plan-Generator/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install langchain openai langchain_community

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "sk-proj-G4OwU0k5Czohsr6kFmwrZDOIq45oy3Zm2K9PBA"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, PositiveInt, PositiveFloat, field_validator


# User Information
class UserProfile(BaseModel):
    name: str
    age: PositiveInt
    gender: str
    weight: PositiveFloat  # kg
    height: PositiveFloat  # cm
    freetime: PositiveInt  # minutes per day
    goal: str
    activity_level: str
    dietary_preferences: str = None  # Optional

    @field_validator("gender")
    @classmethod
    def validate_gender(cls, v: str) -> str:
        if v.lower() not in ["man", "woman", "non-binary", "other"]:
            raise ValueError("Gender must be valid")
        return v.lower()

    def bmi(self):
        return round(self.weight / ((self.height / 100) ** 2), 1)

    def bmr(self):
        if self.gender == "man":
            return round(10 * self.weight + 6.25 * self.height - 5 * self.age + 5, 1)
        elif self.gender == "woman":
            return round(10 * self.weight + 6.25 * self.height - 5 * self.age - 161, 1)
        else:
            return round(10 * self.weight + 6.25 * self.height - 5 * self.age, 1)


# Plan
def create_prompt(profile: UserProfile, task: str) -> str:
    """Creates a very specific and direct prompt for the LLM."""

    # Start building the main part of the prompt
    prompt_content = f"""
You are a professional expert in {task}.
Create a weekly {task} plan for a {profile.age}-year-old {profile.gender} with the following profile:
Weight: {profile.weight} kg | Height: {profile.height} cm | BMI: {profile.bmi()} | BMR: {profile.bmr()}
Activity Level: {profile.activity_level}
Daily free time: {profile.freetime} minutes
Goal: {profile.goal}
"""

    # Conditionally add dietary preferences/restrictions
    if profile.dietary_preferences:
        prompt_content += f"Dietary preferences: {profile.dietary_preferences}\n"

    # Add the instruction for plan delivery
    prompt_content += "\nProvide a detailed day-by-day plan for 6 days."

    # Add the IMPORTANT instructions for the LLM's response format
    prompt_content += f"""

IMPORTANT: Your response MUST NOT include any conversational introduction or preamble. Do not say "Hello," "Certainly," "Here is your plan," or anything similar at the beginning. Your response should start directly with the first day of the plan.

After the entire plan, you MUST add the following exact sentence:
"I hope this meal plan helps you, {profile.name}, on your way to a healthier lifestyle. If you have any specific preferences or changes you would like to make, let me know!"
"""
    # Finally, return the complete constructed prompt string
    return prompt_content


# LLM setup
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)


def generate_plan(profile: UserProfile, task: str) -> str:
    """Generates the plan using the marketing-style prompt."""
    prompt = create_prompt(profile, task)
    return conversation.run(prompt)


if __name__ == "__main__":

    print("🚀 Welcome to your Personal Health & Fitness Coach! 🚀")
    print("-----------------------------------------------------")
    print("Let's craft a plan designed just for you. We'll start with a few quick questions.")

    # Get user information
    try:
        name = input("First, what should I call you? My name is: ")
        age = int(input(f"Great, {name}! Let's talk numbers. What's your age? "))
        gender = input("What's your gender (man/woman)? ")
        weight = float(input("What's your current weight (in kg)? "))
        height = float(input("And your height (in cm)? "))
        freetime = int(input("How many minutes do you have for yourself each day? "))
        goal = input("What's your primary goal (e.g., build muscle, lose weight, maintain health)? ")
        activity_level = input("Describe your activity level (e.g., sedentary, lightly active, active): ")
        dietary_preferences = input("Any dietary preferences or restrictions? (e.g., vegetarian, dairy-free) [Optional]: ") or None

        user = UserProfile(
            name=name,
            age=age,
            gender=gender,
            weight=weight,
            height=height,
            freetime=freetime,
            goal=goal,
            activity_level=activity_level,
            dietary_preferences=dietary_preferences
        )

        # Choose the type of plan
        task_choice = input("\nPerfect! Now, what kind of plan will help you achieve your goals? (meal planning or workout plan): ")

        if task_choice.lower() not in ["meal planning", "workout plan"]:
            print("That's an interesting choice! We'll start with 'meal planning' to lay a solid foundation.")
            task = "meal planning"
        else:
            task = task_choice

        print("\nCrafting your personalized plan... one moment please! ✨")

        plan = generate_plan(user, task=task)
        print("\n--- Your Personalized Plan is Here! ---")
        print(plan)

    except ValueError as e:
        print(f"Oops! It looks like there was an invalid entry: {e}. Please make sure you enter numbers for age, weight, height, and freetime.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        print("Please double-check your API key and internet connection, then try again!")

🚀 Welcome to your Personal Health & Fitness Coach! 🚀
-----------------------------------------------------
Let's craft a plan designed just for you. We'll start with a few quick questions.
First, what should I call you? My name is: Parisa
Great, Parisa! Let's talk numbers. What's your age? 29
What's your gender (man/woman)? woman
What's your current weight (in kg)? 58
And your height (in cm)? 168
How many minutes do you have for yourself each day? 30
What's your primary goal (e.g., build muscle, lose weight, maintain health)? maintain health
Describe your activity level (e.g., sedentary, lightly active, active): lightly active
Any dietary preferences or restrictions? (e.g., vegetarian, dairy-free) [Optional]: dairy-free

Perfect! Now, what kind of plan will help you achieve your goals? (meal planning or workout plan): meal planning

Crafting your personalized plan... one moment please! ✨

--- Your Personalized Plan is Here! ---
Day 1:
- Breakfast: Avocado toast on gluten-free bread wit